# Project : LCA with python

I wiill preform an LCA on the production of smartphone.
After that, a contribution analysis should be conducted, as well as visualising the results.

# 1. Setting Up 
Here we need proper packages, databases to conduct LCA. We will conduct three steps which are  


(1) Importing relevant packages, 

(2) Ecoinvent database

(3) Create a database for the product (based on the excel file)

### 1-1. Importing the Relevant packages

In [ ]:
# import the packages
import bw2data as bd # the data handling brightway package
import bw2io as bi # the data input out package, used to import ecoinvent
import bw2calc as bc # the package that does the math
import bw2analyzer as bwa # used for the iterative calculations
from brightway2 import *
bw2setup()

# Impport basic libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

In [2]:
bd.projects.set_current("myproject")
bd.databases

Databases dictionary with 5 object(s):
	DB_Name
	Smartphone
	biosphere3
	ecoinvent
	ecoinvent 3.91 cutoff

### 1-2. Importing Ecoinvent Database

In [3]:
path = r"C:\Users\Kaayin\Documents\Python SoSe24\MGT001437_project\Code_LCA_new\Code_LCA\ecoinvent 3.9.1_cutoff_ecoSpold02\datasets"
ei = SingleOutputEcospold2Importer(path, "ecoinvent")
ei.apply_strategies()
ei.write_database()

Extracting ecospold2 files:
0% [##############################] 100% | ETA: 00:00:00 | Item ID: fffede58-fed2-5
Total time elapsed: 00:08:11


Title: Extracting ecospold2 files:
  Started: 06/17/2024 22:29:31
  Finished: 06/17/2024 22:37:43
  Total time elapsed: 00:08:11
  CPU %: 50.80
  Memory %: 36.71
Extracted 21238 datasets in 494.26 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_un

OperationalError: cannot VACUUM from within a transaction

### 1-3. Importing LCI data

Bring your Inventory data
Then use this codes so your inventory will be linked to ecoinvent data and biosphere.

In [4]:
# Import the excel LCI for a NMC cells
filepath = r"C:\Users\Kaayin\Documents\Python SoSe24\MGT001437_project\Code_LCA_new\Code_LCA\LCI_Example - 2.xlsx" # path to excel LCI
em_importer = bi.ExcelImporter(filepath) # create importer object

# Check for empty cells in Excel and remove (thanks to Romain Sacchi)
for x in em_importer.data:
        for k, v in list(x.items()):
            if not v:
                del x[k]

em_importer.apply_strategies() # apply strategies to process the excel data

# we link imported activities to one another first, if needed
# on the basis of their name, unit and location
em_importer.match_database(fields=["name", "unit", "location"])

# then we link to ecoinvent for the remaining unlinked technosphere exchanges
# on the basis of their name, reference product, unit and location
em_importer.match_database('ecoinvent 3.91 cutoff',  fields=["reference product", "name", "unit", "location"])

# and then we link emission flows to the biosphere database
em_importer.match_database('biosphere3',  fields=["name", "unit", "categories"])

# shows stats about unlinked exchanged
em_importer.statistics()

Extracted 1 worksheets in 0.11 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 10.08 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
5 datasets
40 exchanges
40 unlinked exchang

(5, 40, 40)

Write another database with activity

In [5]:
em_importer.write_database()
ems = bd.Database('Smartphone') #create object
list([act for act in ems])
India_act = ems.search('India')[0]
Brazil_act = ems.search('Brazil')[0]
Europe_act = ems.search('Europe')[0]
US_act = ems.search('USA')[0]
China_act = ems.search('China')[0]

Writing activities to SQLite3 database:


InvalidExchange: 

# 2. Selecting an Impact Category Method

Due to its common use, we chose to pick an LCA method from the "ReCiPe 2016 v1.03" library. There are many methods that calculate GWP, so we plot preliminary results on IPMSM machine production to compare the differences. In the end, we choose the method "ReCiPe 2016 v1.03, midpoint (I) no LT".

### 2-1. Selecting Method

In [ ]:
us_meth=[met for met in bd.methods if 'ReCiPe 2016 v1.03, midpoint (E) no LT' in met[0] ]
us_meth = us_meth
print(us_meth)

In [ ]:
for method in us_meth:
    impact= method
    unit = bd.methods.get(method).get('unit')
    print(impact)
    print(unit)

# 3. Conduct LCA

In [ ]:
result = []
activity = [India_act, Brazil_act, Europe_act, US_act, China_act]
for act in activity:
    for method in us_meth:
        # perform lca for nmc cells at pack level
        lca = bc.LCA({act:1}, method)
        lca.lci()
        lca.lcia()
        result.append({'name' : act['name'],
                        'Score':lca.score,
                        'Impact Category':method,
                        'unit':bd.methods.get(method).get('unit')})

In [ ]:
result_df = pd.DataFrame(result)
result_df

# 4. Exporting the data

In [ ]:
result_df.to_csv('result.csv', index=False)

print("DataFrame exported to 'result.csv' successfully.")